# 2016년도 대회 
- 2003년부터 2015년까지 샌프란시스코의 수집한 범죄 데이터 제공
- 주어진 데이터로 총 39개의 범죄 유형을 예측 (다중 분류 문제)

## 데이터 불러오기

In [ ]:
!pip install kaggle

In [ ]:
!ls "/content/drive/MyDrive/Colab Notebooks/sf-crime"

In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/sf-crime/*.zip'

In [ ]:
import os 
Data_path = './'
for file in os.listdir(Data_path):
    if 'csv' in file and 'zip' not in file:
        print(file.ljust(30) + str(round(os.path.getsize(file)/1000000, 2)) + 'MB')


In [ ]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.shape, test.shape

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
sample = pd.read_csv('sampleSubmission.csv')
sample.info()

## 탐색적 자료 분석


In [ ]:
# copy
train_df = train.copy()
train_df.shape == train.shape

In [ ]:
# 결측치 확인
def check_na(data):
    isnull_na = (data.isnull().sum() / len(data) * 100)
    data_na = isnull_na.drop(isnull_na[isnull_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing ratio' : data_na,
                                'Data Type': data.dtypes[data_na.index]})
    print("결측치 데이터 컬럼과 건수:\n", missing_data)
check_na(train_df)

In [ ]:
# 중복 데이터 확인
train_df.duplicated().sum()

In [ ]:
# 중복 데이터 제거
print(train_df )
train_df.drop_duplicates(inplace=True)
print(train_df)

In [ ]:
# 범죄 내용과 관련된 두 개의 변수 삭제
train_df.drop(['Descript', 'Resolution'], axis=1, inplace=True)


In [ ]:
train_df.shape

In [ ]:
train_df.head()

In [ ]:
# 상위 5개 출력
train_df['Category'].value_counts()[:5]

- 절도죄, 폭행, 마약 관련 범죄가 많음을 알 수 있음

In [ ]:
# 하위 5개 출력
train_df['Category'].value_counts()[-5:]

- 강요죄, 성범죄, 도박 등이 있는 것을 알 수 있음

In [ ]:
temp = train_df.groupby('DayOfWeek').count().iloc[:, 0]
temp = temp.reindex([
                     'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
])
print(temp)

In [ ]:
# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import numpy as np

fig, ax = plt.subplots(figsize=(10, 6))

ax = sns.barplot(
    temp.index, (temp.values/temp.values.sum()) * 100,
    orient='v'
)
ax.set_title('Incident Rates Rates', fontdict={'fontsize': 16})
ax.set_xlabel('Weekday')
ax.set_ylabel('Incidents (%)')

plt.show()

- 금요일에 가장 많은 범죄가 발생하고, 일요일이 상대적으로 적은 숫자가 관측됨

In [ ]:
# 각각의 범죄 요일별 확인
for idx, data in enumerate(train_df.groupby(['Category'])['DayOfWeek']):
    print(data[0])
    print(round(data[1].value_counts() / data[1].count() * 100, 1))
    print()
    if idx == 1: # 전체를 다 알고 싶으면 if문 주석처리
        break

In [ ]:
temp